In [1]:
%config Completer.use_jedi = False


import pandas as pd
from sqlalchemy import create_engine
from config import *
import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
ath_2016_file = "Resources/athletes_2016.csv"
ath_2016_df = pd.read_csv(ath_2016_file)
ath_2016_df.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [3]:
countries_2016_file = "Resources/countries_2016.csv"
country_2016_df = pd.read_csv(countries_2016_file)
country_2016_df.head()

,country,code,population,gdp_per_capita
0,Afghanistan,AFG,32526562.0,594.323081
1,Albania,ALB,2889167.0,3945.217582
2,Algeria,ALG,39666519.0,4206.031232
3,American Samoa*,ASA,55538.0,NaN
4,Andorra,AND,70473.0,NaN


In [4]:
ath_2021_file = "Resources/athletes_2021.csv"
ath_2021_df = pd.read_csv(ath_2021_file)
clean_ath_2021_df = ath_2021_df.rename(columns={'Name':'name','NOC':'country','Discipline':'sport'})
clean_ath_2021_df.head()

,name,country,sport
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [5]:
medals_2021_file = "Resources/medals_2021.csv"
medals_2021_df = pd.read_csv(medals_2021_file)
medals_2021_df.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,7,3,4,14,2
1,2,Japan,7,1,3,11,3
2,3,People's Republic of China,6,3,6,15,1
3,4,Great Britain,3,2,1,6,7
4,5,Republic of Korea,3,0,4,7,6


In [6]:
medals_2021_df.rename(columns={'Team/NOC':'Country'}, inplace=True)
medals_2021_df = medals_2021_df[['Country','Gold','Silver','Bronze','Total']]
medals_2021_df.rename(columns={'Country':'country','Gold':'gold','Silver':'silver','Bronze':'bronze','Total':'total'},inplace=True)
medals_2021_df.sort_values('total', inplace=True, ascending=False)
medals_2021_df.head()

,country,gold,silver,bronze,total
2,People's Republic of China,6,3,6,15
0,United States of America,7,3,4,14
1,Japan,7,1,3,11
5,ROC,2,5,3,10
8,Italy,1,3,4,8


In [7]:
clean_country_df = country_2016_df.rename(columns = {'code':'nationality'})
clean_country_df.head()

,country,nationality,population,gdp_per_capita
0,Afghanistan,AFG,32526562.0,594.323081
1,Albania,ALB,2889167.0,3945.217582
2,Algeria,ALG,39666519.0,4206.031232
3,American Samoa*,ASA,55538.0,NaN
4,Andorra,AND,70473.0,NaN


In [8]:
merged_2016_df = pd.merge(ath_2016_df, clean_country_df, on="nationality", how="left")
merged_2016_df.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze,country,population,gdp_per_capita
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0,Spain,46418269.0,25831.582305
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0,"Korea, South",50617045.0,27221.524051
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1,Canada,35851774.0,43248.529909
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0,Moldova,3554150.0,1848.061804
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0,New Zealand,4595700.0,37807.967276


In [9]:
clean_ath_2016_df = merged_2016_df[['name','country','sport']]
clean_ath_2016_df.dropna(inplace=True)
clean_ath_2016_df.head()

,name,country,sport
0,A Jesus Garcia,Spain,athletics
1,A Lam Shin,"Korea, South",fencing
2,Aaron Brown,Canada,athletics
3,Aaron Cook,Moldova,taekwondo
4,Aaron Gate,New Zealand,cycling


In [10]:
age_2016_df = merged_2016_df[['country','sex']]
age_2016_df.rename(columns={'sex':'gender'},inplace=True)
age_2016_df.head()

,country,gender
0,Spain,male
1,"Korea, South",female
2,Canada,male
3,Moldova,male
4,New Zealand,male


In [11]:
def fix_date(x):
    if x.year > 2016:
        year = x.year - 100
    else:
        year = x.year
    return datetime.date(year,x.month,x.day)

age_2016_df['dob'] = pd.to_datetime(merged_2016_df['dob'])
age_2016_df.dropna(inplace=True)
age_2016_df['dob'] = age_2016_df['dob'].apply(fix_date)
age_2016_df['average_age'] = 2016 - pd.DatetimeIndex(age_2016_df['dob']).year
age_2016_df.head()

,country,gender,dob,average_age
0,Spain,male,1969-10-17,47
1,"Korea, South",female,1986-09-23,30
2,Canada,male,1992-05-27,24
3,Moldova,male,1991-01-02,25
4,New Zealand,male,1990-11-26,26


In [12]:
avg_age_df = age_2016_df.groupby(['country','gender']).mean()
avg_age_df['average_age'] = avg_age_df['average_age'].astype(int)
avg_age_df.reset_index(inplace=True)
avg_age_df.head()

,country,gender,average_age
0,Afghanistan,female,20
1,Afghanistan,male,27
2,Albania,female,23
3,Albania,male,24
4,Algeria,female,25


In [13]:
medals_2016_df = merged_2016_df.groupby(['country'])[['gold', 'silver', 'bronze']].sum()
medals_2016_df.head()

,gold,silver,bronze
country,,,
Afghanistan,0,0,0
Albania,0,0,0
Algeria,0,2,0
American Samoa*,0,0,0
Andorra,0,0,0


In [14]:
medals_2016_df ['total'] = medals_2016_df['gold'] +  medals_2016_df['silver'] +  medals_2016_df['bronze']
medals_2016_df.sort_values('total', inplace=True, ascending=False)
medals_2016_df.reset_index(inplace=True)
medals_2016_df.head()

,country,gold,silver,bronze,total
0,United States,139,54,71,264
1,Germany,49,44,67,160
2,United Kingdom,64,55,26,145
3,Russia,52,29,34,115
4,China,46,30,37,113


## postgreSQL connection

In [15]:
rds_connection_string = "{}:{}@localhost:5432/olympics_db".format(username,password)

engine = create_engine(f'postgresql://{rds_connection_string}')

In [16]:
engine.table_names()

['athlete_data_2016',
 'athlete_data_2021',
 'medals_2016',
 'medals_2021',
 'average_age_2016']

In [17]:
clean_ath_2016_df

,name,country,sport
0,A Jesus Garcia,Spain,athletics
1,A Lam Shin,"Korea, South",fencing
2,Aaron Brown,Canada,athletics
3,Aaron Cook,Moldova,taekwondo
4,Aaron Gate,New Zealand,cycling
...,...,...,...
11533,Zurian Hechavarria,Cuba,athletics
11534,Zuzana Hejnova,Czech Republic,athletics
11535,di Xiao,China,wrestling
11536,le Quoc Toan Tran,Vietnam,weightlifting


In [18]:
clean_ath_2016_df.to_sql(name='athlete_data_2016', con=engine, if_exists='append', index=False)

In [19]:
clean_ath_2021_df.to_sql(name='athlete_data_2021', con=engine, if_exists='append', index=False)

In [20]:
medals_2021_df.to_sql(name='medals_2021', con=engine, if_exists='append', index=False)

In [21]:
medals_2016_df.to_sql(name='medals_2016', con=engine, if_exists='append', index=False)

In [22]:
avg_age_df.to_sql(name='average_age_2016', con=engine, if_exists='append', index=False)

In [23]:
pd.read_sql_query('select * from athlete_data_2016', con=engine).head()

,id,name,country,sport
0,1,A Jesus Garcia,Spain,athletics
1,2,A Lam Shin,"Korea, South",fencing
2,3,Aaron Brown,Canada,athletics
3,4,Aaron Cook,Moldova,taekwondo
4,5,Aaron Gate,New Zealand,cycling


In [24]:
pd.read_sql_query('select * from athlete_data_2021', con=engine).head()

,id,name,country,sport
0,1,AALERUD Katrine,Norway,Cycling Road
1,2,ABAD Nestor,Spain,Artistic Gymnastics
2,3,ABAGNALE Giovanni,Italy,Rowing
3,4,ABALDE Alberto,Spain,Basketball
4,5,ABALDE Tamara,Spain,Basketball


In [25]:
pd.read_sql_query('select * from medals_2021', con=engine).head()

,country,gold,silver,bronze,total
0,People's Republic of China,6,3,6,15
1,United States of America,7,3,4,14
2,Japan,7,1,3,11
3,ROC,2,5,3,10
4,Italy,1,3,4,8


In [26]:
pd.read_sql_query('select * from medals_2016', con=engine).head()

,country,gold,silver,bronze,total
0,United States,139,54,71,264
1,Germany,49,44,67,160
2,United Kingdom,64,55,26,145
3,Russia,52,29,34,115
4,China,46,30,37,113


In [27]:
pd.read_sql_query('select * from average_age_2016', con=engine).head()

,id,country,gender,average_age
0,1,Afghanistan,female,20
1,2,Afghanistan,male,27
2,3,Albania,female,23
3,4,Albania,male,24
4,5,Algeria,female,25
